In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Data Read & Process") \
    .getOrCreate()




25/06/21 04:46:05 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
df = spark.read.csv("gs://datafor_work/indian_customer_sample.csv", header=True, inferSchema=True)

In [4]:
spark

In [4]:
df.show(5)

+-----------+---------------+--------------------+--------------+-------+-----------------+--------------------+---------+
|customer_id|           name|                city|         state|country|registration_date|               email|is_active|
+-----------+---------------+--------------------+--------------+-------+-----------------+--------------------+---------+
|          1|   Ritvik Issac|              Panvel|       Tripura|  India|       2021-02-24|mamootyhayer@lad.biz|     true|
|          2|   Rania Saxena|       Vijayanagaram|     Jharkhand|  India|       2024-03-08|bhattikismat@yaho...|    false|
|          3|Aniruddh Bhasin|         Mahbubnagar|  Chhattisgarh|  India|       2021-07-15| mchacko@jhaveri.com|     true|
|          4|  Faiyaz Sachar|    Sultan Pur Majra| Uttar Pradesh|  India|       2024-11-12|   vramesh@gmail.com|     true|
|          5|   Neelofar Bir|Sangli-Miraj & Ku...|Andhra Pradesh|  India|       2021-05-29|ayeshasarraf@yaho...|    false|
+-----------+---

In [3]:
df.printSchema()


root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: date (nullable = true)
 |-- email: string (nullable = true)
 |-- is_active: boolean (nullable = true)



In [5]:
df = df.fillna({'city':'Unknown','state':'Unknown','country':'Unknown'})

In [13]:
from pyspark.sql.functions import *


In [10]:
df = df.withColumn('registration_year',year(col('registration_date')))

In [11]:
df.show(5)

+-----------+---------------+--------------------+--------------+-------+-----------------+--------------------+---------+-----------------+
|customer_id|           name|                city|         state|country|registration_date|               email|is_active|registration_year|
+-----------+---------------+--------------------+--------------+-------+-----------------+--------------------+---------+-----------------+
|          1|   Ritvik Issac|              Panvel|       Tripura|  India|       2021-02-24|mamootyhayer@lad.biz|     true|             2021|
|          2|   Rania Saxena|       Vijayanagaram|     Jharkhand|  India|       2024-03-08|bhattikismat@yaho...|    false|             2024|
|          3|Aniruddh Bhasin|         Mahbubnagar|  Chhattisgarh|  India|       2021-07-15| mchacko@jhaveri.com|     true|             2021|
|          4|  Faiyaz Sachar|    Sultan Pur Majra| Uttar Pradesh|  India|       2024-11-12|   vramesh@gmail.com|     true|             2024|
|          5|

In [19]:
unique_Cities = df.select(countDistinct('city')).collect()
unique_Cities[0][0]

316

In [20]:
unique_State = df.select(countDistinct('state')).collect()
unique_State[0][0]

28

In [21]:
df.groupBy('city').count().orderBy(col('count').desc()).show()

+-------------+-----+
|         city|count|
+-------------+-----+
|   Aurangabad|  318|
|    Ghaziabad|  293|
|       Rampur|  199|
|       Bhopal|  197|
|        Eluru|  189|
|  Medininagar|  188|
|    Serampore|  187|
|    Bharatpur|  186|
| Narasaraopet|  185|
|        Sirsa|  184|
|    Bangalore|  184|
|    Raebareli|  182|
|       Chapra|  182|
|  Farrukhabad|  181|
|    Jehanabad|  181|
|Machilipatnam|  181|
|        Bidar|  181|
|       Purnia|  180|
|       Ranchi|  180|
|   Ramagundam|  179|
+-------------+-----+
only showing top 20 rows



In [22]:
df.groupBy('State').count().orderBy(col('count').desc()).show()

+-----------------+-----+
|            State|count|
+-----------------+-----+
|           Punjab| 1870|
|          Gujarat| 1865|
|           Sikkim| 1842|
|         Nagaland| 1831|
|          Haryana| 1829|
|   Madhya Pradesh| 1828|
|            Assam| 1824|
|Arunachal Pradesh| 1813|
|            Bihar| 1813|
|       Tamil Nadu| 1813|
|          Mizoram| 1801|
|           Kerala| 1793|
|      Uttarakhand| 1790|
|          Tripura| 1772|
|        Rajasthan| 1772|
|        Karnataka| 1771|
|        Telangana| 1770|
|     Chhattisgarh| 1769|
|      Maharashtra| 1762|
|        Meghalaya| 1760|
+-----------------+-----+
only showing top 20 rows



In [23]:
df.groupBy('state').pivot('is_active').count().show()

+-----------------+-----+----+
|            state|false|true|
+-----------------+-----+----+
|           Kerala|  912| 881|
|          Manipur|  882| 869|
|              Goa|  870| 887|
| Himachal Pradesh|  877| 870|
|          Gujarat|  968| 897|
|           Sikkim|  891| 951|
|        Meghalaya|  880| 880|
|        Karnataka|  890| 881|
|           Punjab|  989| 881|
|          Mizoram|  922| 879|
|        Jharkhand|  833| 859|
|        Telangana|  874| 896|
|          Tripura|  887| 885|
|         Nagaland|  954| 877|
|     Chhattisgarh|  809| 960|
|Arunachal Pradesh|  870| 943|
|        Rajasthan|  885| 887|
|      Maharashtra|  884| 878|
|            Bihar|  919| 894|
|           Odisha|  888| 854|
+-----------------+-----+----+
only showing top 20 rows



In [31]:
from pyspark.sql.window import Window

In [36]:

window_spec = Window.partitionBy('state').orderBy(col('registration_date').desc())

df = df.withColumn('rank',rank().over(window_spec))\
    .withColumn('dense rank',dense_rank().over(window_spec))\
        .withColumn('row_number',row_number().over(window_spec))
            

In [29]:
df.select('customer_id','name','city','state','rank','dense rank','row_number').show(10)

+-----------+---------------+---------------+--------------+----+----------+----------+
|customer_id|           name|           city|         state|rank|dense rank|row_number|
+-----------+---------------+---------------+--------------+----+----------+----------+
|      34586|     Urvi Saran|    Medininagar|Andhra Pradesh|   1|         1|         1|
|      20491| Kashvi Khurana|      Chinsurah|Andhra Pradesh|   2|         2|         2|
|      14126|Chirag Dhaliwal|       Siliguri|Andhra Pradesh|   3|         3|         3|
|      31182|   Shayak Shere|     Maheshtala|Andhra Pradesh|   3|         3|         4|
|      28172|  Himmat Mangat|          Hosur|Andhra Pradesh|   5|         4|         5|
|      36390|       Pihu Sha|Tiruchirappalli|Andhra Pradesh|   6|         5|         6|
|      13716|      Heer Arya|      Bangalore|Andhra Pradesh|   7|         6|         7|
|      19063|    Miraan Soni|        Dhanbad|Andhra Pradesh|   7|         6|         8|
|      38067|Jivika Upadhyay|   

In [31]:
df_recent_customers = df.filter(col("registration_date") >= '2024-05-18')
df_recent_customers.count()

10991

In [33]:
#oldest and newest customer 
df.groupBy('city').agg(min('registration_date').alias('oldest'),max('registration_date').alias('newest')).show()

+---------------+----------+----------+
|           city|    oldest|    newest|
+---------------+----------+----------+
|        Khandwa|2020-07-14|2025-06-15|
|          Bhind|2020-06-25|2025-06-18|
|     Aurangabad|2020-06-19|2025-06-15|
|          Hosur|2020-06-29|2025-06-15|
|       Uluberia|2020-06-19|2025-06-07|
|         Rampur|2020-06-24|2025-06-12|
|    Bidhannagar|2020-06-23|2025-06-10|
|          Ajmer|2020-06-21|2025-06-02|
|  Karawal Nagar|2020-06-20|2025-06-03|
|        Jalgaon|2020-06-24|2025-06-19|
|       Agartala|2020-06-28|2025-06-19|
|          Korba|2020-06-24|2025-05-08|
|      Chinsurah|2020-06-20|2025-06-18|
|         Tezpur|2020-07-29|2025-06-05|
|       Vadodara|2020-06-24|2025-06-11|
|   Ichalkaranji|2020-06-20|2025-06-14|
|          Sirsa|2020-07-16|2025-06-11|
|Tiruchirappalli|2020-07-02|2025-06-12|
|         Purnia|2020-06-22|2025-06-15|
|      Bhavnagar|2020-07-04|2025-06-14|
+---------------+----------+----------+
only showing top 20 rows



In [34]:
output_path = 'gs://datafor_work/processed_customers'
df.write.mode('overwrite').parquet(output_path)

# Joining and analyzing Customers and Orders

In [8]:
orders_df = spark.read.format('csv').option('header','true').option('inferSchema',True).load("gs://datafor_work/orders_sample_data.csv")


In [9]:
orders_df.show(5)

+--------+-----------+----------+------------+---------+
|order_id|customer_id|order_date|total_amount|   status|
+--------+-----------+----------+------------+---------+
|       1|      49949|2023-09-10|     30667.8|Cancelled|
|       2|       5120|2023-12-14|    33121.87|  Shipped|
|       3|       1167|2024-04-07|    36165.83|  Pending|
|       4|      42589|2023-04-09|      1449.0|  Pending|
|       5|      11125|2025-03-09|     9198.06|  Pending|
+--------+-----------+----------+------------+---------+
only showing top 5 rows



In [10]:
orders_df.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- status: string (nullable = true)



In [11]:
customers_orders_df = df.join(orders_df,'customer_id',"inner")

In [19]:
customers_orders_df.toPandas().head(10)

,customer_id,name,city,state,country,registration_date,email,is_active,order_id,order_date,total_amount,status
0,1,Ritvik Issac,Panvel,Tripura,India,2021-02-24,mamootyhayer@lad.biz,True,13926,2025-04-20,1958.45,Delivered
1,1,Ritvik Issac,Panvel,Tripura,India,2021-02-24,mamootyhayer@lad.biz,True,4495,2024-12-06,19030.65,Delivered
2,4,Faiyaz Sachar,Sultan Pur Majra,Uttar Pradesh,India,2024-11-12,vramesh@gmail.com,True,37807,2024-09-17,25008.51,Shipped
3,4,Faiyaz Sachar,Sultan Pur Majra,Uttar Pradesh,India,2024-11-12,vramesh@gmail.com,True,22386,2022-08-15,42219.43,Delivered
4,5,Neelofar Bir,Sangli-Miraj & Kupwad,Andhra Pradesh,India,2021-05-29,ayeshasarraf@yahoo.com,False,15434,2022-12-24,4053.10,Shipped
5,5,Neelofar Bir,Sangli-Miraj & Kupwad,Andhra Pradesh,India,2021-05-29,ayeshasarraf@yahoo.com,False,3495,2023-11-20,27569.09,Pending
6,7,Darshit Jani,Raiganj,Karnataka,India,2022-07-03,desaizoya@bhasin.com,True,38879,2023-11-18,21586.86,Shipped
7,9,Rania Bajwa,Warangal,Uttar Pradesh,India,2024-05-18,rati73@hotmail.com,False,11648,2022-12-18,49825.15,Delivered
8,13,Samarth Dugar,Tiruvottiyur,Tripura,India,2024-12-31,charvi33@yahoo.com,False,38344,2024-04-19,11185.36,Pending
9,15,Yakshit Iyengar,Ranchi,Mizoram,India,2022-06-23,sumersavant@gokhale.com,True,27646,2025-01-25,19378.31,Pending


#### TOTAL ORDER PER CUSTOMERS

In [23]:
customer_orders_count = customers_orders_df.groupBy('customer_id').count().orderBy(col('count').desc())
customer_orders_count.show(12)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|      43420|    7|
|      44020|    6|
|      21528|    6|
|      23387|    6|
|      31024|    6|
|       1255|    6|
|      23035|    6|
|      46172|    6|
|      35051|    6|
|      19619|    6|
|      26391|    6|
|      42890|    6|
+-----------+-----+
only showing top 12 rows



#### Total Spend Per Customer


In [42]:
customer_total_spend = customers_orders_df.groupBy('customer_id').agg(sum('Total_Amount')).orderBy(col('sum(total_amount)').desc())
                                                                                                   
customer_total_spend.show(10)                                            

+-----------+------------------+
|customer_id| sum(Total_Amount)|
+-----------+------------------+
|        267|         224520.43|
|      34604|196253.28000000003|
|      16653|         195857.53|
|      26352|194770.30000000002|
|      26391|         190916.52|
|      38900|189194.77000000002|
|       6854|         186094.41|
|       3521|184771.25999999998|
|      49610|         183674.87|
|      46172|         181590.14|
+-----------+------------------+
only showing top 10 rows



#### Average Spend per Customer

In [41]:
customer_total_spend_avg = customers_orders_df.groupBy('customer_id').agg(avg('Total_Amount')).orderBy(col('avg(total_amount)').desc())
                                                                                                   
customer_total_spend_avg.show(10)              

+-----------+-----------------+
|customer_id|avg(Total_Amount)|
+-----------+-----------------+
|      41657|         49994.82|
|      44032|         49990.16|
|       2196|         49989.94|
|      48970|         49985.82|
|      48231|         49985.64|
|      10176|         49984.87|
|      31127|         49979.35|
|      13448|         49969.25|
|      39820|         49969.22|
|      24154|         49967.64|
+-----------+-----------------+
only showing top 10 rows



In [28]:
order_status_count = customers_orders_df.groupBy('status').count()
order_status_count.show()

+---------+-----+
|   status|count|
+---------+-----+
|Cancelled| 9880|
|Delivered|10172|
|  Shipped|10071|
| Returned|10007|
|  Pending| 9870|
+---------+-----+



#### Orders By Month

In [29]:
orders_by_month = customers_orders_df.withColumn('order_month',month(col('order_date')))\
    .groupby('order_month')\
        .count()\
            .orderBy('order_month')

orders_by_month.show()

+-----------+-----+
|order_month|count|
+-----------+-----+
|          1| 4154|
|          2| 3912|
|          3| 4241|
|          4| 4145|
|          5| 4287|
|          6| 4166|
|          7| 4247|
|          8| 4317|
|          9| 4061|
|         10| 4225|
|         11| 4057|
|         12| 4188|
+-----------+-----+



In [43]:
customer_total_spend.printSchema() , customer_orders_count.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- sum(Total_Amount): double (nullable = true)

root
 |-- customer_id: integer (nullable = true)
 |-- count: long (nullable = false)



(None, None)

#### Finding Customers with High Order Frequency but Low Spend

In [44]:
customer_spend_vs_orders = customer_orders_count.join(customer_total_spend,'customer_id','inner')\
    .orderBy(col('count').desc(),col('sum(total_amount)'))

customer_spend_vs_orders.show(5)

+-----------+-----+------------------+
|customer_id|count| sum(Total_Amount)|
+-----------+-----+------------------+
|      43420|    7|147295.08000000002|
|      49919|    6|          78464.38|
|       4416|    6| 84123.23000000001|
|      28680|    6| 86722.12999999999|
|       2436|    6|         100395.06|
+-----------+-----+------------------+
only showing top 5 rows



In [45]:
output_path = 'gs://datafor_work/processed_customer_and_orders'
df.write.mode('overwrite').parquet(output_path)